Assignment: Flowers Recognition <br>
Dataset Description:<br>

This dataset contains 4242 images of flowers.<br>
The data collection is based on the data flicr, google images, yandex images.<br>
You can use this datastet to recognize plants from the photo.<br>

Attribute Information:<br>
The pictures are divided into five classes: chamomile, tulip, rose, sunflower, dandelion.<br>
For each class there are about 800 photos. Photos are not high resolution, about 320x240 pixels. <br>
<b>Also explore how to resize images in tensorflow and then resize all the images to a same size. </b> <br>
This is a Multiclass Classification Problem.<br>




WORKFLOW : <br>
Load Data <br>
Split into 60 and 40 ratio.<br>
Encode labels.<br>
Create Model<br>
Compilation Step (Note : Its a Multiclass Classification problem , select loss , metrics according to it)<br>
Train the Model.<br>
If the model gets overfit tune your model by changing the units , No. of layers , epochs , add dropout layer or add Regularizer according to the need .<br>
Prediction should be > 85%<br>
Evaluation Step<br>
Prediction<br>




Data : <br>
https://drive.google.com/file/d/1-OX6wn5gA-bJpjPNfSyaYQLz-A-AB_uj/view?usp=sharing

In [290]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
import os


In [291]:
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

# data visualisation and manipulation
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#preprocess.
from keras.preprocessing.image import ImageDataGenerator

#dl libraraies
import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# specifically for cnn
from keras.layers import Dropout, Flatten,Activation
 
from PIL import Image

In [292]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [293]:
cd /content/drive/MyDrive/PIAIC-ASSIGNMENTS/DLAssignments/Data

/content/drive/MyDrive/PIAIC-ASSIGNMENTS/DLAssignments/Data


In [432]:
import pathlib

FLOWER_DAISY_DIR='/content/drive/MyDrive/PIAIC-ASSIGNMENTS/DLAssignments/Data/flowers/daisy'
FLOWER_SUNFLOWER_DIR='/content/drive/MyDrive/PIAIC-ASSIGNMENTS/DLAssignments/Data/flowers/sunflower'
FLOWER_TULIP_DIR='/content/drive/MyDrive/PIAIC-ASSIGNMENTS/DLAssignments/Data/flowers/tulip'
FLOWER_DANDI_DIR='/content/drive/MyDrive/PIAIC-ASSIGNMENTS/DLAssignments/Data/flowers/dandelion'
FLOWER_ROSE_DIR='/content/drive/MyDrive/PIAIC-ASSIGNMENTS/DLAssignments/Data/flowers/rose'


In [433]:
DAISY_File_Array=os.listdir(FLOWER_DAISY_DIR)
SUNFLOWER_File_Array=os.listdir(FLOWER_SUNFLOWER_DIR)
TULIP_File_Array=os.listdir(FLOWER_TULIP_DIR)
DANDELION_File_Array=os.listdir(FLOWER_DANDI_DIR)
ROSE_File_Array=os.listdir(FLOWER_ROSE_DIR)


print(len(DAISY_File_Array),len(SUNFLOWER_File_Array),len(TULIP_File_Array),len(DANDELION_File_Array),len(ROSE_File_Array))

769 734 984 1052 784


In [ ]:
OneImage=np.asarray(Image.open("/content/drive/MyDrive/PIAIC-ASSIGNMENTS/DLAssignments/Data/flowers/daisy/8120563761_ed5620664f_m.jpg"))
print(OneImage.dtype)
print("--------------1-----------------")
print(OneImage)

SumImage=OneImage[::,::,0]+OneImage[::,::,1]+OneImage[::,::,2]
print("Sum Image",SumImage)
OneImage =(OneImage[::,::,0]+OneImage[::,::,1]+OneImage[::,::,2])/3 ;

print("--------------2-----------------")
print(OneImage)
OneImage = resize(OneImage, (50,50),anti_aliasing=True)
print("--------------3-----------------")
print(OneImage)    
OneImage=np.ravel(OneImage)
print("--------------4-----------------")
print(OneImage)


In [588]:
from skimage import data, color
from skimage.transform import  resize

def loadFlowerData(FlowerArray,Path):
  All_Image_Array=np.zeros(2500);
  for flower in FlowerArray:
    SingleImage=np.asarray(Image.open(Path+"/"+flower)).astype('float64')
    SingleImage =(SingleImage[:,:,0]+SingleImage[:,:,1]+SingleImage[:,:,2])/3 ;
    SingleImage = resize(SingleImage, (50,50),anti_aliasing=True)
    SingleImage=np.ravel(SingleImage)
    All_Image_Array=np.vstack((np.atleast_2d(All_Image_Array), np.atleast_2d(SingleImage)))
  return All_Image_Array[1:]

  # 
  #   SingleImage=np.asarray(Image.open(Path+"/"+flower))
  #   SingleImage=(SingleImage[::,::,0]+SingleImage[::,::,1]+SingleImage[::,::,2])/3
  #   All_Image_Array=np.append(All_Image_Array,SingleImage, axis=0)
  # return All_Image_Array

In [589]:
Daisy_ImgArr=loadFlowerData(DAISY_File_Array,FLOWER_DAISY_DIR)
print(Daisy_ImgArr.shape)

(769, 2500)


In [590]:
SunFlower_ImgArr=loadFlowerData(SUNFLOWER_File_Array,FLOWER_SUNFLOWER_DIR)


In [591]:
Tulip_ImgArr=loadFlowerData(TULIP_File_Array,FLOWER_TULIP_DIR)


In [592]:
Dandelion_ImgArr=loadFlowerData(DANDELION_File_Array,FLOWER_DANDI_DIR)

In [593]:
Rose_ImgArr=loadFlowerData(ROSE_File_Array,FLOWER_ROSE_DIR)


In [594]:
print("Daisy Shape : ",Daisy_ImgArr.shape,"\n","Sunflower Shape: ",SunFlower_ImgArr.shape,"\n","Tulip Shape: ",Tulip_ImgArr.shape,"\n","Dandelion Shape: ",Dandelion_ImgArr.shape,"\n","Rose Shape: ",Rose_ImgArr.shape)

Daisy Shape :  (769, 2500) 
 Sunflower Shape:  (734, 2500) 
 Tulip Shape:  (984, 2500) 
 Dandelion Shape:  (1052, 2500) 
 Rose Shape:  (784, 2500)


In [595]:
Daisy_Complete=np.insert(Daisy_ImgArr, 2500, values='0', axis=1)
Sunflower_Complete=np.insert(SunFlower_ImgArr, 2500, values='1', axis=1)
Tulip_Complete=np.insert(Tulip_ImgArr, 2500, values='2', axis=1)
Dandelion_Complete=np.insert(Dandelion_ImgArr, 2500, values='3', axis=1)
Rose_Complete=np.insert(Rose_ImgArr, 2500, values='4', axis=1)



In [596]:
All_Flowers=np.vstack((Daisy_Complete,Sunflower_Complete,Tulip_Complete,Dandelion_Complete,Rose_Complete))
print(All_Flowers.shape)

(4323, 2501)


In [597]:
import sklearn
Shuffled_All_Flowers = sklearn.utils.shuffle(All_Flowers)


In [598]:
sample_flowers=Shuffled_All_Flowers[:,:2500]
results_flowers=Shuffled_All_Flowers[:,-1]
print("result/sample size",results_flowers.shape,sample_flowers.shape)



result/sample size (4323,) (4323, 2500)


In [599]:
sample_flowers_train, sample_flowers_test, results_flowers_train, results_flowers_test = train_test_split(sample_flowers, results_flowers, test_size=0.4, random_state=42)

print("train result/sample size",sample_flowers_train.shape,results_flowers_train.shape)
print("test result/sample size",sample_flowers_test.shape,results_flowers_test.shape)


train result/sample size (2593, 2500) (2593,)
test result/sample size (1730, 2500) (1730,)


# Network architect


In [688]:
from keras import models,layers,regularizers
model=Sequential()
model.add(Dense(512,activation='relu',kernel_regularizer=regularizers.l2(0.001),input_shape=(2500,)))
model.add(layers.Dropout(0.3))
model.add(layers.Dense(100, activation='relu',kernel_regularizer=regularizers.l2(0.001)))
model.add(layers.Dropout(0.3))
model.add(Dense(5,activation='softmax'))

# Compilation Step

In [692]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(),loss=tf.keras.losses.categorical_crossentropy ,metrics=['accuracy'])

In [693]:
sample_flowers_train[0]

array([40.37331379, 42.86613654, 42.24698541, ..., 99.41893808,
       95.42227166, 93.73703616])

In [694]:
normalized_sample_flowers_train=sample_flowers_train.astype('float32')/255
normalized_sample_flowers_test=sample_flowers_test.astype('float32')/255

In [695]:
from tensorflow.keras.utils import to_categorical
categorized_results_flowers_test=to_categorical(results_flowers_test)
categorized_results_flowers_train=to_categorical(results_flowers_train)


In [696]:
print(sample_flowers_train.shape,categorized_results_flowers_train.shape)

(2593, 2500) (2593, 5)


In [699]:
history=model.fit(normalized_sample_flowers_train,categorized_results_flowers_train,epochs=1000,batch_size=500)

Epoch 1/1000
6/6 [==============================] - 0s 65ms/step - loss: 1.5629 - accuracy: 0.3498
Epoch 2/1000
6/6 [==============================] - 0s 63ms/step - loss: 1.5454 - accuracy: 0.3459
Epoch 3/1000
6/6 [==============================] - 0s 68ms/step - loss: 1.4775 - accuracy: 0.3683
Epoch 4/1000
6/6 [==============================] - 0s 63ms/step - loss: 1.4660 - accuracy: 0.3722
Epoch 5/1000
6/6 [==============================] - 0s 63ms/step - loss: 1.5016 - accuracy: 0.3598
Epoch 6/1000
6/6 [==============================] - 0s 63ms/step - loss: 1.5213 - accuracy: 0.3521
Epoch 7/1000
6/6 [==============================] - 0s 64ms/step - loss: 1.4758 - accuracy: 0.3806
Epoch 8/1000
6/6 [==============================] - 0s 67ms/step - loss: 1.4755 - accuracy: 0.3791
Epoch 9/1000
6/6 [==============================] - 0s 64ms/step - loss: 1.5188 - accuracy: 0.3498
Epoch 10/1000
6/6 [==============================] - 0s 62ms/step - loss: 1.5338 - accuracy: 0.3351
Epoch 11/

In [698]:
test_loass,test_acc=model.evaluate(normalized_sample_flowers_test,categorized_results_flowers_test)

55/55 [==============================] - 0s 6ms/step - loss: 1.7145 - accuracy: 0.3069
